# <div style="text-align: center"><font color='#dc2624' face='微软雅黑'>计量信用组合经济资本</font></div>

## <font color='#dc2624' face='微软雅黑'>目录</font><a name='toc'></a>
### 1. [**<font color='#dc2624' face='微软雅黑'>默顿模型</font>**](#1)
1. [<font color='#2b4750' face='微软雅黑'>模型简介</font>](#1.1)
2. [<font color='#2b4750' face='微软雅黑'>违约事件</font>](#1.2)
3. [<font color='#2b4750' face='微软雅黑'>违约距离</font>](#1.3)
4. [<font color='#2b4750' face='微软雅黑'>资产建模</font>](#1.4)

### 2. [**<font color='#dc2624' face='微软雅黑'>间接法</font>**](#2)
1. [<font color='#2b4750' face='微软雅黑'>数学推导</font>](#2.1)
2. [<font color='#2b4750' face='微软雅黑'>案例分析</font>](#2.2)

### 3. [**<font color='#dc2624' face='微软雅黑'>直接法</font>**](#3)
1. [<font color='#2b4750' face='微软雅黑'>数学推导</font>](#3.1)
2. [<font color='#2b4750' face='微软雅黑'>案例分析</font>](#3.2)

---

<div style="text-align: right"> Conceptually, a
firm’s equity stake is essentially equivalent to a call option on its assets with the
strike price equal to its debt obligations.</div>
<div style="text-align: right"> — Robert C.Merton</div>

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import norm, mvn
import numpy.linalg as nla
import scipy.optimize as spo

np.random.seed(1031)
mpl.rcParams['font.sans-serif'] = ['Microsoft YaHei']
mpl.rcParams['axes.unicode_minus'] = False

# <font color='#dc2624' face='微软雅黑'>1. 默顿模型</font><a name='1'></a>
[<font color='black' face='微软雅黑'>回到目录</font>](#toc)
### <font color='#2b4750' face='微软雅黑'>1.1 模型简介</font><a name='1.1'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#1)

默顿模型 (Merton's model) 描述公司资产价值 (asset value)，它在任何时点 $t$ 等于其权益值和负债值之和，其中 $t ∈ [0, T]$

<br/>
<font color='blue'>
\begin{equation}
A_t = E_t + D_t
\end{equation}
</font>

其中 $A_t, E_t, D_t$ 代表公司在 $t$ 时点的资产值 (asset)、权益值 (equity) 和负债值 (debt)。

- 当 $A_t > D_t$，公司没破产
- 当 $A_t \leq D_t$，公司破产

公司权益实际上就是公司把欠得债偿还之后剩下的价值，因此

<br/>
<font color='blue'>
\begin{equation}
E_t = \max(A_t - D_t, 0) = (A_t - D_t)^+
\end{equation}
</font>

这样其实公司权益是一个看涨期权

- **标的价格** (underlying) 是公司资产值
- **行权价格** (strike) 是公司负债值

### <font color='#2b4750' face='微软雅黑'>1.2 违约事件</font><a name='1.2'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#1)

当 $A_T \leq K$，公司破产等价于违约，定义 $\tau$ 为第一次公司资产小于负债的**时点**，那么有以下等价关系：

<br>
<font color='blue'>
\begin{equation}
\{\tau \leq T\} = \{A_t \leq K\}
\end{equation}
</font>

定义违约事件 $\mathcal{D} = \{\tau \leq T\}$，那么违约概率是

<br>
<font color='blue'>
\begin{equation}
\mathbb{P}\left(\mathcal{D}\right) = \mathbb{E}\left[\mathbb{I}_{\mathcal{D}}\right] = \mathbb{E}\left[\mathbb{I}_{\{\tau \leq T\}}\right] = \mathbb{P}\left(\tau \leq T\right) = \mathbb{P}\left(A_t \leq K\right)
\end{equation}
</font>

下图画出某资产价格走势图以及违约点 (default point)。

<img src="default point.png" style="width:500px;height:300px;">

### <font color='#2b4750' face='微软雅黑'>1.3 违约距离</font><a name='1.3'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#1)

在真实世界测度下，首先用一个随机微分方程 (stochastic differential equation, SDE) 对资产价值建模：

<br>
<font color='blue'>
\begin{equation}
 \frac{dA_t}{A_t} = \mu dt + \sigma dW_t
\end{equation}
</font>

根据伊藤定理解出公司在时点 $T$ 的价值

<font color='blue'>
\begin{equation}
A_{T} = A_{t}e^{\left(\mu-\frac{\sigma^2}{2}\right)(T-t)+\sigma\left(W_{T}-W_{t}\right)}
\end{equation}
</font>

违约概率表达式可以进一步具体化

<br>
<font color='blue'>
\begin{equation}
\begin{aligned}
\mathbb{P}\left(A_T \leq K\right) &= \mathbb{P}\left(A_{t}e^{\left(\mu-\frac{\sigma^2}{2}\right)(T-t)+\sigma\left(W_{T}-W_{t}\right)} \leq K\right) \\
&= \mathbb{P}\left(\frac{W_T-W_t}{\sqrt{T-t}} \leq -\frac{\ln\left(\frac{A_t}{K}\right) + \left(\mu-\frac{1} {2}\sigma^2\right)\left(T-t\right)}{\sigma\sqrt{T-t}}\right) \\
&= \mathbb{P}\left(\frac{W_T-W_t}{\sqrt{T-t}} \leq -DD_t\right) \\
&= \frac{1}{2\pi} \int_{-DD_t}^{\infty}e^{-\frac{u^2}{2}}du \\
&= \Phi\left( -DD_t \right) 
\end{aligned}
\end{equation}
</font>

其中 $\Phi(\cdot)$ 是标准正态随机变量的**累计分布函数** (cumulative distribution function, CDF)，而 $DD_t$ 称作**违约距离** (distance-to-default, DD)，其表达式为：

<br>
<font color='blue'>
\begin{equation}
 DD_t = -\frac{\ln\left(\frac{A_t}{K}\right) + \left(\mu-\frac{1}{2}\sigma^2\right)\left(T-t\right)}{\sigma\sqrt{T-t}}
\end{equation}
</font>

违约距离量化了公司离违约有多远，即公司里违约有多少个标准差。

<img src="whole picture.png" style="width:640px;height:400px;">

### <font color='#2b4750' face='微软雅黑'>1.4 资产建模</font><a name='1.4'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#1)

### <font color='black' face='微软雅黑'>单个公司</font>

对于上市公司，它的资产值是观察不到的，市场看到的股价其实是它的权益值。权益值和资产值有联系。在风险中性 (risk-neutral) 世界测度下，资产价格的 SDE 为 (注意漂移项从 $\mu$ 变成了 $r$)：

<br>
<font color='blue'>
\begin{equation}
 \frac{dA_t}{A_t} = rdt + \sigma dW_t
\end{equation}
</font>

根据上面**权益是资产的看涨期权**的结论，我们有

<br>
<font color='blue'>
\begin{equation}
E_t = e^{-r(T-t)} \cdot \mathbb{E}\left[\left( A_T-K\right)^+\right] = A_t\Phi(d_+) - e^{-r(T-t)}K\Phi(d_{-}) 
\end{equation}
</font>

其中

<br>
<font color='blue'>
\begin{equation}
d_{\pm} = \frac{1}{\sigma\sqrt{T-t}}\ln\left(\frac{A_te^{r(T-t)}}{K}\right)\pm\frac{\sigma\sqrt{T-t}}{2}
\end{equation}
</font>

### <font color='black' face='微软雅黑'>多个公司</font>

假设用 $N$ 个公司，对于第 $n$ 公司的资产建模如下，分别写成其资产价格在真实世界和风险中性世界下的 SDE：

<br>
<font color='blue'>
\begin{equation}
 \frac{dA_{n,t}}{A_{n,t}} = \mu_{n}dt + \sigma_n dW_{n,t}, \quad \text{真实世界}
\end{equation}
\begin{equation}
 \frac{dA_{n,t}}{A_{n,t}} = rdt + \sigma_n dW_{n,t}, \quad \text{风险中性世界}
\end{equation}
</font>

解出第 $n$ 个公司在时点 $T$ 的价值 (用真实世界的 SDE)
<font color='blue'>
\begin{equation}
A_{n,T} = A_{n,t}e^{\left(\mu_{n}-\frac{\sigma_n^2}{2}\right)(T-t)+\sigma_n\left(W_{n,T}-W_{n,t}\right)}
\end{equation}
</font>

以及第 $n$ 个公司股票在时点 $t$ 的价值 (用风险中性世界的 SDE)

<br>
<font color='blue'>
\begin{equation}
E_{n,t} = e^{-r(T-t)} \cdot \mathbb{E}\left[\left( A_{n,T}-K_n\right)^+\right] = A_{n,t}\Phi(d_+) - e^{-r(T-t)}K_n\Phi(d_{-}) 
\end{equation}
</font>

其中

<br>
<font color='blue'>
\begin{equation}
d_{\pm} = \frac{1}{\sigma_n\sqrt{T-t}}\ln\left(\frac{A_{n,t}e^{r(T-t)}}{K_n}\right)\pm\frac{\sigma_n\sqrt{T-t}}{2}
\end{equation}
</font>

默顿模型中有两种实现方式：

- **间接法** (indirect approach)：假设每个公司的违约概率已知，反推出它们的违约距离 $DD_n$ 和违约点 $K_n$。 
- **直接法** (direct apporach)：根据每个公司的资产负债表 (balance-sheet) 信息来计算违约距离 $DD_n$。

第二和三章分别来看间接法和直接法。

# <font color='#dc2624' face='微软雅黑'>2. 间接法</font><a name='2'></a>
[<font color='black' face='微软雅黑'>回到目录</font>](#toc)
### <font color='#2b4750' face='微软雅黑'>2.1 数学推导</font><a name='2.1'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#2)

如果已知资产价格 $A_{n,t}$、收益率 $\mu_n$、波动率 $\sigma_n$ 和违约概率 $p_n$，我们可以退出违约对应的行权价值 $K_n$。

<br/>
<font color='blue'>
\begin{equation}
p_n = \mathbb{P}\left(A_{n,T} \leq K_n\right) = \Phi\left( -\frac{\ln\left(\frac{A_{n,t}}{K_n}\right) + \left(\mu_n-\frac{1}{2}\sigma_n^2\right)\left(T-t\right)}{\sigma_n\sqrt{T-t}} \right) = \Phi\left( -DD_n \right)
\end{equation}
</font>

反推出 $K_n$ 和 $DD_{n}$ 的表达式：

<br/>
<font color='blue'>
\begin{equation}
K_n = A_{n,t}e^{\left(\mu_n-\frac{1}{2}\sigma_n^2 \right)\left(T-t\right) + \Phi^{-1}\left(p_n \right)\sigma_n \sqrt{T-t}}
\end{equation}
\begin{equation}
DD_{n} = -\Phi^{-1}\left(p_n \right)
\end{equation}
</font>

计算 $K_n$ 和 $DD_{n}$ 的代码如下：

In [ ]:
def getK( p, A, mu, sigma, dt ):
    return A * np.exp((mu -0.5*sigma**2)*dt+ norm.ppf(p)*sigma*np.sqrt(dt))

def getDD( A, K, mu, sigma, dt ):
    return (np.log(A/K) + (mu -0.5*sigma**2)*dt)/(sigma*np.sqrt(dt))

研究资产波动率 $\sigma_n$、违约概率 $p_n$、违约障碍 $K_n$ 和违约距离 $DD_n$ 之间的关系。

In [ ]:
A = np.array( [100] * 10 )
mu = np.array( [0.01] * 10 )
T = np.array( [0.25] * 10 )
sigma = np.array( [0.15]*5 + [0.3]*5 )
p = np.array( [0.001, 0.005, 0.01, 0.02, 0.05] * 2 )

In [ ]:
K = getK( p, A, mu, sigma, T )
DD = getDD( A, K, mu, sigma, T )

In [ ]:
mny_fmt = '${0:,.2f}'
num_fmt = '{0:,.2f}'
pct_fmt = '{0:,.2f}%'

In [ ]:
data = np.vstack( [A, mu*100, sigma*100, p*100, K, DD] ).T
label = ['Asset', 'Drift', 'Volatility', 'PD', 'Strike', 'DD']
df = pd.DataFrame( data = data, columns=label )
df.style.format({"Asset": mny_fmt, 
                 "Drift": pct_fmt, 
                 "Volatility": pct_fmt,
                 "PD":pct_fmt,
                 "Strike": mny_fmt, 
                 "DD": num_fmt})

,Asset,Drift,Volatility,PD,Strike,DD
0,$100.00,1.00%,15.00%,0.10%,$79.29,3.09
1,$100.00,1.00%,15.00%,0.50%,$82.41,2.58
2,$100.00,1.00%,15.00%,1.00%,$83.96,2.33
3,$100.00,1.00%,15.00%,2.00%,$85.70,2.05
4,$100.00,1.00%,15.00%,5.00%,$88.37,1.64
5,$100.00,1.00%,30.00%,0.10%,$62.36,3.09
6,$100.00,1.00%,30.00%,0.50%,$67.36,2.58
7,$100.00,1.00%,30.00%,1.00%,$69.93,2.33
8,$100.00,1.00%,30.00%,2.00%,$72.85,2.05
9,$100.00,1.00%,30.00%,5.00%,$77.45,1.64


从上表可以观察到三条规律：

1. 当违约概率相同时，波动率越**大**，违约障碍离资产价格越**远**。 (0-5 行，1-6 行，2-7 行，3-8 行，4-9 行)

2. 当波动率相同时，违约概率越**大**，违约障碍离资产价格越**近**。(0 到 4 行，5 到 9 行)

3. 违约距离和违约概率一一对应，和波动率无关。(0-5 行，1-6 行，2-7 行，3-8 行，4-9 行)

再回到最早推违约概率的公式

<br>
<font color='blue'>
\begin{equation}
\mathbb{P}\left( A_{n,T} \leq K_n\right) 
= \mathbb{P}\left(\frac{W_{n,T}-W_{n,t}}{\sqrt{T-t}} \leq DD_{n}\right) 
= \mathbb{P}\left( \xi_n \leq \Phi^{-1}\left(p_n \right)\right) 
\end{equation}
</font>

其中 $\xi$ 服从 $n$ 维标准正态分布。

<br>
<font color='blue'>
\begin{equation}
\underbrace{\left[\begin{array}{c}
\xi_{1} \\
\xi_{2} \\
\vdots \\
\xi_{N}
\end{array}\right]}_{\xi} =
\begin{array}{c}
\frac{1}{\sqrt{T-t}}
\end{array}\left[\begin{array}{c}
W_{1, T}-W_{1, t} \\
W_{2, T}-W_{2, t} \\
\vdots \\
W_{N, T}-W_{N, t}
\end{array}\right]
\sim \mathcal{N}\left(\left[\begin{array}{c}
0 \\
0 \\
\vdots \\
0
\end{array}\right], \underbrace{\left[\begin{array}{cccc}
1 & \rho_{12} & \cdots & \rho_{1 N} \\
\rho_{21} & 1 & \cdots & \rho_{2 N} \\
\vdots & \vdots & \ddots & \vdots \\
\rho_{N 1} & \rho_{N 2} & \cdots & 1
\end{array}\right]}_{\Omega_{N}}\right)
\end{equation}
</font>

### <font color='#2b4750' face='微软雅黑'>2.2 案例分析</font><a name='2.2'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#2)

首先考虑一个样本组合，它包含 100 个不同的借贷人，有如下三个假设：

1.	组合的总规模为 1000，意味着平均每个借贷人的敞口 (exposure) 为 10。
2.	实际敞口是根据**韦伯分布** (Weibull)模拟得出，范围从小于 1 到 50。
3.	债务人的平均无条件违约概率根据**卡方分布** (chi-square) 模拟得出，为 1％。

违约概率和敞口已经模拟好，接下来只需读取其 `.npy` 格式的文件。

In [ ]:
dpFile = os.getcwd() + '\\defaultProbabilties.npy'
expFile = os.getcwd() + '\\exposures.npy'

信用组合的损失 $L$ 可写成每个债务人的损失 $L_n$ 之和:

<br>
<font color='blue'>
\begin{equation}
L = \sum_{n=1}^{N} L_{n}=\sum_{n=1}^{N} u_{n}\cdot \left(1-R_{n}\right) \cdot \mathbb{I}_{\mathcal{D}_{n}}
\end{equation}
</font>

其中
- <font color='blue'>$\mathbb{I}_{\mathcal{D}_{n}}=\left\{\begin{array}{ll}
1: & \text { 在 } T \text { 之前违约, 概率为 } p_{n} \\
0: & \text { 存活到 } T, \text { 概率为 } 1-p_{n}
\end{array}\right.$</font> 是违约指示函数 (default indicator)
- <font color='blue'>$u_{n}$</font> 是 EAD，exposure-at-default 的缩写，代表违约时暴露
- <font color='blue'>$1-R_{n}$</font> 是 LGD，loss-give-default 的缩写，代表违约时损失率，而 LGD 等于 1 减去恢复率 (recovery rate)

当 EAD 和 LGD 之间没有相关性时，为了之后推导简化，我们可以将上式写成

<br>
<font color='blue'>
\begin{equation}L=\sum_{n=1}^{N} c_{n} \mathbb{I}_{\mathcal{D}_{n}}, \quad \text { 其中 } c_{n}=u_{n} \cdot\left(1-R_{n}\right)\end{equation}
</font>

In [ ]:
c = np.load(expFile)  
p = np.load(dpFile)
N = len(c)
M = 1000000
q = [0.95, 0.97, 0.99, 0.995, 0.999, 0.9997, 0.9999]
q_style = [str(i*100) +'%' for i in q]
money_fmt = '${0:,.2f}'
number_fmt = '{0:,.2f}'

信用风险建模的重点是构建信用组合的违约损失分布（default loss distribution），该分布描述了组合中潜在违约产生的所有可能损失。

有了损失分布，我们还可以估算组合的

- **预期损失**（Expected Loss, EL）
- **意外损失**（Unexcpeted Loss, UL）
- **风险价值**（Value-at-Risk, VaR）
- **期望损失**（Expected Shortfall, ES）
- **经济资本**（Economic Capital, EC）

预期损失 EL 计量的是因为违约的平均损失，银行会为 EL 拿出一部分钱作为储备。

风险价值 VaR 和期望损失 ES 都属于极端损失，VaR 是在一段时间内在 q 概率下组合损失的最大值，在信用风险中 q 通常设定为 99%, 99.5%, 99.9% 等。而 ES 是大于 VaR 的损失的均值。通俗来讲，VaR 量化坏情况，而 ES 量化的是如果坏情况发生那么到底有多坏。

有些资料把 VaR 和 ES 归属到意外损失 UL，有些资料把损失变量的波动率定义成 UL，这个也无对错之分，我更偏好于后者。而且我觉得把 VaR 和 ES 称为极端损失也更贴切些。

按照上面的叫法，经济资本可定义为极端损失和预期损失的差，可看成是银行为了极端事件而准备的额外资本（rainy-day fund）。

首先需要明晰损失 $L$ 是一个随机变量，既然有随机性，那么我们可以计算 $L$ 的统计指标：

    EL = mean[L]
    UL = stdev[L]
    VaR = quantile(L, q)
    ES = mean(L|L>=VaR)
    EC = VaR - EL

所以找到 $L$ 的分布最重要。将上面所有信息可视化成下图：

<img src="loss distribution.png" style="width:600px;height:300px;">

两个函数即可搞定用间接法默顿模型计算所有重要指标：

- `indirectMerton_LD()`：给定 N 个借贷人和 M 个情景，模拟出损失分布
- `risk_measure()`: 给定损失分布 `LD`，分位数 `q` 和插值方法 `flag`，计算出 `EL, UL, VaR, ES`

先看如何生成损失分布，难点就是如何生成相关 (correlated) 随机变量，用 Cholesky 分解即可。

<img src="Cholesky.png" style="width:800px;height:220px;">

然后构建违约指示函数：

<font color='blue'>
\begin{equation}
\mathbb{I}_{\mathcal{D}_{n}}=\left\{\begin{array}{ll}
1: & \text { 在 } T \text { 之前违约, 概率为 } \mathbb{P}\left( \xi_n \leq \Phi^{-1}\left(p_n \right)\right)  \\
0: & \text { 存活到 } T, \text { 概率为 } 1-\mathbb{P}\left( \xi_n \leq \Phi^{-1}\left(p_n \right)\right) 
\end{array}\right.
\end{equation}
</font>

对 $n = 1, 2, …, N$ 和 $m = 1, 2, …, M$, 我们模拟出 NM 个违约指标。剩下的操作就简单了，对于第 $m$ 个模拟情境，计算出组合损失

<br>
<font color='blue'>
\begin{equation}L^{(m)}=\sum_{n=1}^{N} c_{n} \mathbb{I}_{\mathcal{D}_{n}}^{(m)}\end{equation}
</font>

In [ ]:
def indirectMerton_LD( N, M, p, c, corr_mat, q, details=0 ):
    C = nla.cholesky(corr_mat)         # (N,N)
    Z = np.random.normal(0,1,[N,M])    # (N,M)
    xi = C @ Z                         # (N,M)
    K = np.tile( norm.ppf(p),(M,1) )   # (M,N)
    DI = 1*np.less(xi.T, K)            # (M,N)
    LD = np.sort( np.dot(DI,c) )       # (M,)
    if details == 0:
        return LD
    else:
        return LD, DI

有了损失分布，计算 EL, UL, VaR, ES 很容易。对于 EL 和 UL 根据均值和方差的定义来计算它们：

<br>
<font color='blue'>
\begin{equation}\overline{\mathbb{E}[L]}=\frac{1}{M} \sum_{m=1}^{M} L^{(m)}\end{equation}
\begin{equation}\overline{\mathrm{V}[L]}=\frac{1}{M-1} \sum_{m=1}^{M}\left(L^{(m)}-\overline{\mathbb{E}[L]}\right)^{2}\end{equation}
</font>

要计算 VaR 和 ES，我们首先需要对 $L^{(1)}, L^{(2)}, …, L^{(M)}$ 按升序排序，即排序后 L^{(1)} 最小而 L^{(M)} 最大。要计算 $VaR_q$，当 q = 1%，我们计算索引 $M_q$，在返回对应该索引的损失值 $L(M_q)$。如果 $M_q$ 不是整数，假设 M = 250, q = 99%，那么 $M_q$ = 247.5 不是整数，有两种方法返回合理的损失值：

1.	向上取整 $[M_q]$。这时 [247.5] = 248，返回 $L^{(248)}$
2.	找相邻索引，再做线性插值。247.5 的相邻索引为 247 和 248，那么在 $L^{(247)}$ 和 $L^{(248)}$ 线性插值。

第一种方法更保守（因为返回一个更大的损失值）；第二种方法更精确（因为返回一个内插值），不同系统商实现的方式不同，但第二种更常见。

<br>
<font color='blue'>
\begin{equation}\overline{\operatorname{VaR}_{q}[L]}=\left\{\begin{array}{cl}
L^{([M_q])}, & \text { 向上取整 } \\
([M_q]- M_q) L^{([M_q]-1)}+(M_q+1-[M_q]) L^{([M_q])}, & \text { 线性内插 }
\end{array}\right.\end{equation}
</font>

计算出 $VaR_q$ 后，再计算 $ES_q$ 也非常简单。

<br>
<font color='blue'>
\begin{equation}\overline{\mathrm{ES}_{q}[L]}=\frac{1}{M-[M_q]} \sum_{m=[M_q]}^{M} L^{(m)}\end{equation}
</font>

计算 ES 不需要像计算 VaR 那样分情况，因为不管什么情况，$L[M_q]$ 总是第一个大于 VaR 的损失值。那么总共大于 VaR 的损失有 $M – [M_q]$ 个，将它们求个平均就可以了。下面一图胜千言。

<img src="VaR ES.png" style="width:650px;height:180px;">

In [ ]:
def risk_measure( LD, q, flag='nearest' ):
    EL, UL = np.mean(LD), np.std(LD)
    VaR, ES = np.zeros([len(q)]), np.zeros([len(q)])
    M = len(LD)
    for n in range(0,len(q)):
        i = q[n]*(M-1)
        idx = np.ceil(i).astype(int)
        if flag.lower() == 'nearest':
            VaR[n] = LD[idx]
        else:
            VaR[n] = (idx-i)*LD[idx-1] + (i+1-idx)*LD[idx]
        ES[n] = np.mean( LD[idx:M-1] )        
    
    return EL, UL, VaR, ES

上面的 `risk_measure()` 是个通用函数，只需要传入损失分布 `LD`、分位数 `q` 和插值方式 `flag` 就能返回 `EL, UL, VaR, ES`。

用 `indirectMerton_simulation()` 函数整合前面两个函数，使得代码看起来更有结构。

In [ ]:
def indirectMerton_simulation( N, M, p, c, corr_mat, q, return_LD=False ):
    LD = indirectMerton_LD( N, M, p, c, corr_mat, q )
    EL, UL, VaR, ES  = risk_measure( LD, q )
    output = (EL, UL, VaR, ES, LD) if return_LD else (EL, UL, VaR, ES)
    return output

In [ ]:
rho = 0.2
corr_mat = rho*np.ones((N,N)) + np.diag((1-rho)*np.ones(N))

In [ ]:
EL, UL, VaR, ES = indirectMerton_simulation( N, M, p, c, corr_mat, q )

In [ ]:
df1_G = pd.DataFrame( data=np.vstack((VaR, ES)).T,
                       index=q_style,
                       columns=['VaR', 'ES'] )
df1_G.style.format(money_fmt)

,VaR,ES
95.0%,$44.21,$67.10
97.0%,$55.37,$78.96
99.0%,$80.18,$106.67
99.5%,$97.89,$125.45
99.9%,$141.54,$172.34
99.97%,$178.11,$211.23
99.99%,$213.16,$247.50


In [ ]:
df2_G = pd.DataFrame( data=np.hstack((EL,UL)), 
                       columns=['Value'],
                       index=['Expected Loss', 'Unexpected Loss'])
df2_G.style.format(money_fmt)

,Value
Expected Loss,$9.18
Unexpected Loss,$17.77


# <font color='#dc2624' face='微软雅黑'>3. 直接法</font><a name='3'></a>
[<font color='black' face='微软雅黑'>回到目录</font>](#toc)
### <font color='#2b4750' face='微软雅黑'>3.1 数学推导</font><a name='3.1'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#3)

在直接法中，违约事件写成 $\{ A_{n,T} \leq K_n \}$，要计算违约概率，我们需要比较在 $T$ 时点资产和负债的大小。整理需要的市场数据后发现有五个数据都是未知的。

| 名称      | 符号        |状态 |
|-----------|:------------:|:---:|
| 公司个数  | $N$          |已知 |
| 时点      | $t,T$        |已知 |
| 头寸      | $c_n$        |已知 |
| 违约概率  | $p_n$        |已知 |
| 资产价格  | $A_{n,t}$    |**未知** |
| 资产收益率| $\mu_n$      |**未知** |
| 资产波动率| $\sigma_n$   |**未知** |
| 资产相关性| $\rho_{n,m}$ |**未知** |
| 违约阈值  | $K_n$        |**未知** |

核心数据还是要知道 $T$ 时点 $N$ 个公司的资产价值，用列向量 $\mathbf{A}_T$ 表示：

<br/>
<font color='blue'>
\begin{equation}
\mathbf{A}_T = 
    \begin{bmatrix} A_{1,T} \\ A_{2,T} \\ \cdots \\ A_{N,T} \end{bmatrix} = 
    \begin{bmatrix} A_{1,t}e^{\left(\mu_{1}-\frac{\sigma_1^2}{2}\right)(T-t)+\sigma_1\left(W_{1,T}-W_{1,t}\right)} \\ A_{2,t}e^{\left(\mu_{2}-\frac{\sigma_2^2}{2}\right)(T-t)+\sigma_2\left(W_{2,T}-W_{2,t}\right)} \\ \cdots \\ A_{N,t}e^{\left(\mu_{N}-\frac{\sigma_N^2}{2}\right)(T-t)+\sigma_N\left(W_{N,T}-W_{N,t}\right)}\end{bmatrix}
\end{equation}
</font>

其中每一个 $A_{n,t}$ 都服从对数正态分布，对应的随机微分方程为：

<br/>
<font color='blue'>
\begin{equation}
 \frac{dA_{n,t}}{A_{n,t}} = \mu_{n}dt + \sigma_n dW_{n,t}
\end{equation}
</font>

为了推出或校正出上表的未知量，我们需要的大量数学推导，虽然麻烦，但是思路极其清晰：

1. 推出 <font color='blue'>$A_{n,t}$</font> 的期望 <font color='blue'>$\mathbb{E}\left[A_{n,t}\right]$</font> 
2. 推出 <font color='blue'>$A_{n,t}$</font> 的方差 <font color='blue'>$\mathbb{V}\left[A_{n,t}\right]$</font> 
3. 推出 <font color='blue'>$A_{n,t}$</font> 和 <font color='blue'>$A_{m,t}$</font> 的协方差 <font color='blue'>$\mathbb{COV}\left[A_{n,t}, A_{m,t}\right]$</font> 
4. 推出 <font color='blue'>$\mathbb{I}_{\mathcal{D}_{n}}$</font> 和 <font color='blue'>$\mathbb{I}_{\mathcal{D}_{n}}$</font> 的协方差 <font color='blue'>$\rho\left[ \mathbb{I}_{\mathcal{D}_{n}}, \mathbb{I}_{\mathcal{D}_{n}} \right]$</font> 

下面数学公式较多，但推导上都基于正态随机变量**距生成函数** (moment generating function, MGF) 定理而来。

---
当 <font color='red'>$X \sim \mathcal{N}(0,1)$</font> 和 <font color='red'>$Y \sim \mathcal{N}(\mu,\sigma^2)$</font>，它们的 MGF 为
<br/>
<font color='red'>
\begin{equation}M_{X}(t)=e^{\frac{1}{2} t^{2}}, \quad M_{Y}(t)=e^{\mu t\frac{1}{2} +\sigma t^{2}}\end{equation}
</font>

证明如下：

<br/>
<font color='red'>
\begin{equation}
M_{X}(t)=\mathbb{E}\left[e^{tX}\right]=\int_{-\infty}^{+\infty} e^{tx} \cdot \phi(x) dx=e^{\frac{1}{2} t^2} \cdot \int_{-\infty}^{+\infty} \phi(x-t) dx=e^{\frac{1}{2} t^2} \cdot \int_{-\infty}^{+\infty} \phi(z) dz=e^{\frac{1}{2} t^{2}}
\end{equation}
</font>

由于 <font color='red'>$Y = \mu + \sigma X$</font>,

<font color='red'>
\begin{equation}
M_{Y}(t)=\mathbb{E}\left[e^{tY}\right] = \mathbb{E}\left[e^{t\mu + t\sigma X}\right] = e^{t\mu}\mathbb{E}\left[e^{t\sigma X}\right] = e^{t\mu} \cdot e^{\frac{1}{2} \sigma^2t^{2}} = e^{\mu t+\frac{1}{2} \sigma^2t^{2}} 
\end{equation}
</font>

写出其更通用的结果，将 MGF 写成期望和方差的形式。

<font color='red'>
\begin{equation}
M_{Y}(t) = \text{exp}\left(\mathbb{E}\left[Y\right] t + \frac{t^{2}}{2}\mathbb{V}\left[Y\right]\right)
\end{equation}
</font>

---

### <font color='black' face='微软雅黑'>$A_{n,T}$ 的期望</font>

<br/>
<font color='blue'>
\begin{equation}
\begin{aligned}
\mathbb{E}_t\left[A_{n,T}\right] 
    &= \mathbb{E}_t\left[A_{n,t}e^{\left(\mu_{n}-\frac{\sigma_n^2}{2}\right)(T-t)+\sigma_n\left(W_{n,T}-W_{n,t}\right)}\right] \\ 
    &= A_{n,t}e^{\left(\mu_{n}-\frac{\sigma_n^2}{2}\right)(T-t)}\cdot\mathbb{E}_t\left[e^{\sigma_n\left(W_{n,T}-W_{n,t}\right)}\right] \\
    &= A_{n,t}e^{\left(\mu_{n}-\frac{\sigma_n^2}{2}\right)(T-t)}\cdot e^{\frac{\sigma_n^2}{2}(T-t)} \\
    & = A_{n,t}e^{\mu_n(T-t)}
\end{aligned}
\end{equation}
</font>

In [ ]:
def get_mean( mu, dt, A ):
    return A*np.exp(mu*dt)

### <font color='black' face='微软雅黑'>$A_{n,T}$ 的方差</font>

<br/>
<font color='blue'>
\begin{equation}
\begin{aligned}
\mathbb{V}_t\left[A_{n,T}\right] &= \mathbb{E}_t\left[A_{n,T}^2\right] - \mathbb{E}_t\left[A_{n,T}\right]^2 \\ 
    & = \mathbb{E}_t\left[A_{n,t}^2e^{\left(2\mu_{n}-\sigma_n^2\right)(T-t)+2\sigma_n\left(W_{n,T}-W_{n,t}\right)}\right] - A_{n,t}^2e^{2\mu_n(T-t)} \\
    & = A_{n,t}^2e^{(2\mu_n+\sigma_n^2)(T-t)} - A_{n,t}^2e^{2\mu_n(T-t)} \\
    & = A_{n,t}^2e^{2\mu_n(T-t)}\left(e^{\sigma_n^2(T-t)} - 1\right) \\
\end{aligned}
\end{equation}
</font>

In [ ]:
def get_var( mu, sigma, dt, A, K ):
    return A**2*np.exp(2*mu*dt)*(np.exp((sigma**2)*dt)-1)

### <font color='black' face='微软雅黑'>$A_{n,T}$ 和 $A_{m,T}$ 的协方差</font>

<br>
<font color='blue'>
\begin{equation}
\begin{aligned}
\mathbb{COV}_t\left[A_{n,T}, A_{m,T}\right] &= \mathbb{E}_t\left[A_{n,T}A_{m,T}\right] - \mathbb{E}_t\left[A_{n,T}\right]\mathbb{E}_t\left[A_{m,T}\right] \\
    & = \mathbb{E}_t\left[A_{n,T}A_{m,T}\right] - A_{n,t}e^{\mu_n(T-t)}\cdot A_{m,t}e^{\mu_m(T-t)} \\
    & = \mathbb{E}_t\left[A_{n,t}e^{\left(\mu_{n}-\frac{\sigma_n^2}{2}\right)(T-t)+\sigma_n\left(W_{n,T}-W_{n,t}\right)} \cdot A_{m,t}e^{\left(\mu_{m}-\frac{\sigma_m^2}{2}\right)(T-t)+\sigma_m\left(W_{m,T}-W_{m,t}\right)}\right] - A_{n,t}e^{\mu_n(T-t)}\cdot A_{m,t}e^{\mu_m(T-t)} \\
    & = A_{n,t}A_{m,t}e^{\left(\mu_{n}+\mu_{m}-\frac{\sigma_n^2+\sigma_m^2}{2}\right)(T-t)}\mathbb{E}_t\left[e^{\sigma_n\left(W_{n,T}-W_{n,t}\right)} \cdot e^{\sigma_m\left(W_{m,T}-W_{m,t}\right)}\right] - A_{n,t}e^{\mu_n(T-t)}\cdot A_{m,t}e^{\mu_m(T-t)} \\
    & = A_{n,t}A_{m,t}e^{\left(\mu_{n}+\mu_{m}-\frac{\sigma_n^2+\sigma_m^2}{2}\right)(T-t)}e^{\left(\frac{\sigma_n^2+\sigma_m^2}{2}+\rho_{n,m}\sigma_n\sigma_m\right)(T-t)} - A_{n,t}e^{\mu_n(T-t)}\cdot A_{m,t}e^{\mu_m(T-t)} \\
    & = A_{n,t}A_{m,t}e^{(\mu_n+\mu_m)(T-t)}\left(e^{\rho_{n,m}\sigma_n\sigma_m(T-t)} - 1\right) \\
\end{aligned}
\end{equation}
</font>

In [ ]:
def get_cov( A, B, muA, muB, sigmaA, sigmaB, rho, dt ):   
    return A*B*np.exp((muA+muB)*dt)*(np.exp(sigmaA*sigmaB*rho*dt)-1)

### <font color='black' face='微软雅黑'>资产相关性：$A_{n,T}$ 和 $A_{m,T}$ 的相关性</font>

<br>
<font color='blue'>
\begin{equation}
\begin{aligned}
\rho_t\left[A_{n,T}, A_{m,T}\right] 
    &= \frac{\mathbb{COV}_t\left[A_{n,T}, A_{m,T}\right]}{\sqrt{\mathbb{V}_t\left[A_{n,T}\right]\mathbb{V}_t\left[A_{m,T}\right]}} \\
    &= \frac{A_{n,t}A_{m,t}e^{(\mu_n+\mu_m)(T-t)}\left(e^{\rho_{n,m}\sigma_n\sigma_m(T-t)} - 1\right)}{\sqrt{A_{n,t}^2e^{2\mu_n(T-t)}\left(e^{\sigma_n^2(T-t)} - 1\right)}\sqrt{A_{m,t}^2e^{2\mu_m(T-t)}\left(e^{\sigma_m^2(T-t)} - 1\right)}} \\
    &=\frac{e^{\rho_{n,m}\sigma_n\sigma_m(T-t)} - 1}{\sqrt{\left(e^{\sigma_n^2(T-t)} - 1\right)\left(e^{\sigma_m^2(T-t)} - 1\right)}}
\end{aligned}
\end{equation}
</font>

In [ ]:
def get_assetcorr( sigmaA, sigmaB, rho, dt ):   
    return (np.exp(sigmaA*sigmaB*rho*dt)-1)/np.sqrt((np.exp((sigmaA**2)*dt)-1)*(np.exp((sigmaB**2)*dt)-1))

### <font color='black' face='微软雅黑'>违约相关性：$\mathbb{I}_{\mathcal{D}_n}$ 和 $\mathbb{I}_{\mathcal{D}_m}$ 的相关性</font>

<br/>
<font color='blue'>
\begin{equation}
\begin{aligned}
\rho_t\left[\mathbb{I}_{\mathcal{D}_n}, \mathbb{I}_{\mathcal{D}_m}\right] 
    &= \frac{\mathbb{P}_t\left(\mathcal{D}_n \cap \mathcal{D}_m\right) - \mathbb{P}_t\left(\mathcal{D}_n\right)\mathbb{P}_t\left(\mathcal{D}_m\right)}{\sqrt{\mathbb{P}_t\left(\mathcal{D}_n\right)\left(1-\mathbb{P}_t\left(\mathcal{D}_n\right)\right)} 
\sqrt{\mathbb{P}_t\left(\mathcal{D}_m\right)\left(1-\mathbb{P}_t\left(\mathcal{D}_m\right)\right)}} \\
    &= \frac{\Phi_2\left(-DD_{n}, DD_{m}; \rho_{n,m}\right) - \Phi\left(-DD_{n}\right)\Phi\left(-DD_{m}\right)}{\sqrt{\Phi\left(-DD_{n}\right)\Phi\left(-DD_{m}\right)\left(1-\Phi\left(-DD_{n}\right)\right)\left(1-\Phi\left(-DD_{m}\right)\right)}}
\end{aligned}
\end{equation}
</font>

In [ ]:
def get_defaultcorr( A, B, muA, muB, sigmaA, sigmaB, rho, dt, KA, KB):
    DD_A = getDD( A, KA, muA, sigmaA, dt )
    DD_B = getDD( B, KB, muB, sigmaB, dt )  
    pA = norm.cdf(-DD_A)
    pB = norm.cdf(-DD_B)
    pAB, err = mvn.mvnun( np.array([-100, -100]), np.array([-DD_A, -DD_B]),
                          np.array([0, 0]), np.array([[1,rho],[rho,1]]) )   
    return (pAB-pA*pB) / np.sqrt(pA*pB*(1-pA)*(1-pB))

### <font color='red' face='微软雅黑'>违约相关性不是资产相关性！</font>

下表总结本节的所有变量的数学符号和公式。

<img src="Table.PNG" style="width:800px;height:540px;">

### <font color='#2b4750' face='微软雅黑'>3.2 案例分析</font><a name='3.2'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#3)

下表列出 6 个借贷人的信息。

| 借贷人    | 1    |2    |3    |4    |5    |6|
|-----------|:------------:|:---:|:---:|:---:|:---:|:---:|
| 评级  | BBB | AAA | AA | BBB | AA | AAA |
| 头寸 $c_n$  | 10  |10  |10  |10  |10  |10  |
| 夏普比例  $S_n$ | 0.05 | 0.06 | 0.07 | 0.06 | 0.06 | 0.1 |
| 股票价格  $E_{n,t}$ | 56 | 93 | 75 | 75 | 62 | 105 |
| 股票收益率 $\mu_{E_n}$ |4.1% | 3.8% | 4.4% | 4.7% | 3.6% | 5.1% | 
| 股票波动率 $\sigma_{E_n}$ |56.4% | 42.3% | 45.3% | 57.9% | 46.5% | 43.0% |
| 违约阈值 $K_n$  | 70 | 69 | 67 | 92 | 55 | 78 |

股票收益率 <font color='blue'>$\mu_{E_n}$</font> 和波动率 <font color='blue'>$\sigma_{E_n}$</font> 都是根据股价的时间序列计算得到的。

In [ ]:
c = np.array([10, 10, 10, 10, 10, 10])
DE = np.array([1.238, 0.748, 0.892, 1.229, 0.892, 0.738])
SR = np.array([0.05, 0.06, 0.07, 0.06, 0.06, 0.1])
E = np.array([56, 93, 75, 75, 62, 105])
muE = np.array([0.041, 0.038, 0.044, 0.047, 0.036, 0.051])
sE = np.array([0.564, 0.423, 0.453, 0.579, 0.465, 0.43])
K = np.array([70, 69, 67, 92, 55, 78])
(r, T) = (0.01, 1)

股票价格是市场可见的，其收益率和波动率也可以用时间序列算出；但是资产价格和其波动率是不可见的，必须通过两个关系式反推出来：

<br>
<font color='blue'>
\begin{equation}
\begin{aligned}
& G_{1}(A_{n,t}, \sigma_{n}) = A_{n,t} \Phi\left(d_{+}\right)-e^{-r(T-t)} K_{n} \Phi\left(d_{-}\right)-E_{n,t}=0 \\
& G_{2}(A_{n,t}, \sigma_{n}) = \sigma_nA_{n,t} \Phi\left(d_{+}\right)-\sigma_{E_n}E_{n,t}=0
\end{aligned}
\end{equation}
</font>

上面式子 <font color='blue'>$G_{1}(A_{n,t},\sigma_{n})$</font> 实际上就是 Black-Scholes 公式，而式子 <font color='blue'>$G_{2}(A_{n,t},\sigma_{n})$</font> 是用伊藤定理推出 <font color='blue'>$dE_{n,t}$</font>

<br>
<font color='blue'>
\begin{equation}
d E_{n,t}=\left(r \Phi\left(d_{+}\right)+\frac{\sigma_{n} \phi\left(d_{+}\right)}{2 \sqrt{T-t}}\right) A_{n,t} dt+\Phi\left(d_{+}\right) \sigma_{n} A_{n,t} dW_{n,t}
\end{equation}
</font>

然后匹配扩散项
<br>
<font color='blue'>
\begin{equation}
\Phi\left(d_{+}\right)\sigma_{n}A_{n,t} = \sigma_{E_n}E_{n,t}
\end{equation}
</font>

最后求一个优化问题得到 $(A_{n,t}^*, \sigma_{n}^*)$：

<br>
<font color='blue'>
\begin{equation}
\min_{A_{n,t},\sigma_{n}} G_{1}^2(A_{n,t}, \sigma_{n}) + G_{2}^2(A_{n,t},\sigma_{n})
\end{equation}
</font>

In [ ]:
def BS(A, K, r, T, sigma):
    # Black-Scholes Call 
    v = sigma*np.sqrt(T)
    d1 = np.log(A/K)/v + 0.5*v
    d2 = d1 - v
    return A*norm.cdf(d1) - np.exp(-r*T)*K*norm.cdf(d2)

def BSD(A, K, r, T, sigma):
    # Black-Scholes Call Delta
    v = sigma*np.sqrt(T)
    d1 = np.log(A/K)/v + 0.5*v
    return norm.cdf(d1)

def f(x, E, K, r, T, sE):
    G1 = BS(x[0], K, r, T, x[1]) - E
    G2 = x[0]*x[1]*BSD(x[0], K, r, T, x[1]) - sE*E
    return G1**2 + G2**2

In [ ]:
A = []
sA = []

for iE, iK, isE in zip(E, K, sE):
    xstar = spo.minimize(f, (iE,isE), args=(iE,iK,r,T,isE), method='powell')
    A.append(xstar.x[0])
    sA.append(xstar.x[1])

A = np.array(A)
sA = np.array(sA)

### <font color='black' face='微软雅黑'>资产价格</font>

In [ ]:
A

array([125.22447183, 161.31209927, 141.32813908, 165.95732847,
       116.44691671, 182.22194535])

### <font color='black' face='微软雅黑'>资产波动率</font>

In [ ]:
sA

array([0.25421946, 0.24390713, 0.2405505 , 0.26403528, 0.2477821 ,
       0.2478229 ])

### <font color='black' face='微软雅黑'>资产收益率</font>

In [ ]:
muA = r + sA*SR
muA

array([0.02271097, 0.02463443, 0.02683853, 0.02584212, 0.02486693,
       0.03478229])

### <font color='black' face='微软雅黑'>违约距离</font>

In [ ]:
DD = (np.log(A/K)+(muA-0.5*sA**2)*T)/(sA*np.sqrt(T))
DD

array([2.25006323, 3.46084014, 3.09414477, 2.20018671, 3.00373303,
       3.44032284])

### <font color='black' face='微软雅黑'>实际违约概率</font>

In [ ]:
real_prob = norm.cdf(-DD)
real_prob

array([0.01222247, 0.00026925, 0.00098691, 0.01389683, 0.00133345,
       0.00029051])

### <font color='black' face='微软雅黑'>风险中性违约概率</font>

In [ ]:
RN_prob = norm.cdf(-(np.log(A/K)+(r-0.5*sA**2)*T)/(sA*np.sqrt(T)))
RN_prob

array([0.0139012 , 0.0003359 , 0.00124669, 0.01616984, 0.0016214 ,
       0.00041841])

根据上面的公式和代码，我们可以得到下表结果。

| 借贷人    | 1    |2    |3    |4    |5    |6|
|-----------|:------------:|:---:|:---:|:---:|:---:|:---:|
| 违约距离 $DD_n$  | 2.25  | 3.46  | 3.09  | 2.20  | 3.00  | 3.44  |
| 实际违约概率 $\mathbb{P}_t(\mathcal{D}_n)$ |1.222% | 0.027% | 0.099% | 1.379% | 0.133% | 0.029% | 
| 风险中性违约概率 $\mathbb{Q}_t(\mathcal{D}_n)$ |1.390% | 0.034% | 0.125% | 1.617% | 0.162% | 0.042% | 
| 资产价格  $A_{n,t}$ | 125 | 161 | 141 | 166 | 116 | 182 |
| 资产收益率 $\mu_n$ |2.3% | 2.5% | 2.7% | 2.6% | 2.5% | 3.5% | 
| 资产波动率 $\sigma_n$ |25.4% | 24.4% | 24.1% | 26.4% | 24.8% | 24.8% | 

计算出资产波动率后可以计算资产相关性系数，假设不同资产价格的布朗运动之间的相关性为 0.2，即 $\rho_{n,m}=\rho(W_n,W_m)=0.2$。

In [ ]:
nA = len(A)
rho = 0.2
rho_asset = np.ones((nA,nA))

for i in np.arange(nA):
    for j in np.arange(i+1,nA):
        assetcorr = get_assetcorr(sA[i], sA[j], rho, T)
        rho_asset[i,j], rho_asset[j,i] = assetcorr, assetcorr

label = ['公司' + str(i+1) for i in np.arange(nA)]
pd.DataFrame(rho_asset, index=label, columns=label).style.format('{0:,.3f}')

,公司1,公司2,公司3,公司4,公司5,公司6
公司1,1.000,0.195,0.195,0.195,0.195,0.195
公司2,0.195,1.000,0.195,0.195,0.195,0.195
公司3,0.195,0.195,1.000,0.195,0.195,0.195
公司4,0.195,0.195,0.195,1.000,0.195,0.195
公司5,0.195,0.195,0.195,0.195,1.000,0.195
公司6,0.195,0.195,0.195,0.195,0.195,1.000


接着还可以计算违约相关性系数，发现它比资产相关性系数小很多。

In [ ]:
rho_default = np.ones((nA,nA))

for i in np.arange(nA):
    for j in np.arange(i+1,nA):
        defaultcorr = get_defaultcorr( A[i], A[j], muA[i], muA[j], sA[i], sA[j], rho, T, K[i], K[j])
        rho_default[i,j], rho_default[j,i] = defaultcorr, defaultcorr
        
pd.DataFrame(rho_default, index=label, columns=label).style.format('{0:,.3f}')

,公司1,公司2,公司3,公司4,公司5,公司6
公司1,1.000,0.007,0.012,0.028,0.013,0.008
公司2,0.007,1.000,0.004,0.008,0.004,0.003
公司3,0.012,0.004,1.000,0.012,0.006,0.004
公司4,0.028,0.008,0.012,1.000,0.014,0.008
公司5,0.013,0.004,0.006,0.014,1.000,0.004
公司6,0.008,0.003,0.004,0.008,0.004,1.000


和间接法一样，直接法也是先用蒙特卡洛模拟出组合的损失分布，然后再计算 `EL, UL, VaR, ES`。

In [ ]:
N = len(c)
M = 1000000
q = [0.95, 0.97, 0.99, 0.995, 0.999, 0.9997, 0.9999]
q_style = [str(i*100) +'%' for i in q]
money_fmt = '${0:,.2f}'
number_fmt = '{0:,.2f}'

In [ ]:
def directMerton_LD( N, M, A, K, T, muA, sA, c, corr_mat, q, details=0 ):
    C = nla.cholesky(corr_mat)             # (N,N)
    eps = np.random.normal(0,1,[N,M])      # (M,N)
    Z = C @ eps                            # (N,M)
    v = np.tile(sA*np.sqrt(T), (M,1))      # (M,N)
    A_T = A*np.exp((muA-0.5*v**2)*T+v*Z.T) # (M,N)
    K = np.tile(K, (M,1))                  # (M,N)
    DI = 1*np.less(A_T, K)                 # (M,N)
    LD = np.sort( np.dot(DI,c) )           # (M,)
    if details == 0:
        return LD
    else:
        return LD, DI

In [ ]:
def directMerton_simulation( N, M, A, K, T, muA, sA, c, corr_mat, q, return_LD=False ):
    LD = directMerton_LD( N, M, A, K, T, muA, sA, c, corr_mat, q )
    EL, UL, VaR, ES  = risk_measure( LD, q )
    output = (EL, UL, VaR, ES, LD) if return_LD else (EL, UL, VaR, ES)
    return output

In [ ]:
rho = 0.2
corr_mat = rho*np.ones((N,N)) + np.diag((1-rho)*np.ones(N))

In [ ]:
EL, UL, VaR, ES = directMerton_simulation( N, M, A, K, T, muA, sA, c, corr_mat, q )

In [ ]:
df1_G = pd.DataFrame( data=np.vstack((VaR, ES)).T,
                       index=q_style,
                       columns=['VaR', 'ES'] )
df1_G.style.format(money_fmt)

,VaR,ES
95.0%,$0.00,$5.77
97.0%,$0.00,$9.61
99.0%,$10.00,$10.86
99.5%,$10.00,$11.72
99.9%,$10.00,$18.61
99.97%,$20.00,$20.77
99.99%,$20.00,$22.32


In [ ]:
df2_G = pd.DataFrame( data=np.hstack((EL,UL)), 
                       columns=['Value'],
                       index=['Expected Loss', 'Unexpected Loss'])
df2_G.style.format(money_fmt)

,Value
Expected Loss,$0.29
Unexpected Loss,$1.73
